# Process Lost Data

## Read Dataset & Find Lost Data

In [1]:
import numpy as np
import pandas as pd
import collections
import datetime
from read_table import *

In [2]:
table = read_table()

In [3]:
for i in range(1, 11):
    print(table[str(i)].loc[279, ["volume"]]==0)

volume    True
Name: 279, dtype: bool
volume    True
Name: 279, dtype: bool
volume    True
Name: 279, dtype: bool
volume    True
Name: 279, dtype: bool
volume    True
Name: 279, dtype: bool
volume    True
Name: 279, dtype: bool
volume    True
Name: 279, dtype: bool
volume    True
Name: 279, dtype: bool
volume    False
Name: 279, dtype: bool
volume    False
Name: 279, dtype: bool


## Add Features

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [ ]:
for i in range(1, 11):
    table[str(i)]["daily_diff_rise"] = (table[str(i)].close - table[str(i)].open)/table[str(i)].open
    table[str(i)]["upper_diff_rise"] = (table[str(i)].high - table[str(i)].open)/table[str(i)].open
    table[str(i)]["lower_diff_rise"] = (table[str(i)].open - table[str(i)].low)/table[str(i)].open
    table[str(i)]["extrm_vol_ratio"] = (table[str(i)].high-table[str(i)].low)/table[str(i)].volume
    table[str(i)]["daily_vol_ratio"] = (table[str(i)].close - table[str(i)].open)/table[str(i)].volume

In [ ]:
table["1"].head()

In [ ]:
time = []
for i in range(1, 11):
    time.append(table[str(i)].time)
np.array(time)

In [ ]:
date_list = pd.date_range('2019/1/8', '2020/3/26')
date_list = pd.Series(date_list.format())
temp = []
for j in date_list:
    demp = []
    demp.append(j.split('-')[0])
    demp.append(str(int(j.split('-')[1])))
    demp.append(str(int(j.split('-')[2])))
    temp.append("/".join(demp))
date_list = np.array(temp)
date_list

In [ ]:
df = get_table()

In [ ]:
df = table.copy()

In [ ]:
for i in range(1, 11):
    df[str(i)].set_index(["time"], inplace=True)

In [ ]:
df["3"].head()

In [ ]:
import matplotlib.pyplot as plt
values = df["1"].values[:df["1"].shape[0]-1, :]
groups = list(range(df["1"].shape[1]))
i = 1
plt.figure(dpi=100)
for group in groups:
    plt.subplot(len(groups),1, i)
    plt.plot(values[:, group])
    plt.title(df["1"].columns[group], y=0.5, loc='right')
    i+=1
plt.show()

In [ ]:
df["1"].index

In [ ]:
# integer encode direction
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
values[:, 4] = encoder.fit_transform(values[:,4])

# ensure all data is float
values = values.astype('float32')

# normalize features
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(-1, 1))
scaled = scaler.fit_transform(values)

In [ ]:
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    # feature amounts
    n_vars = 1 if type(data) is list else data.shape[1]
    
    df2 = pd.DataFrame(data)
    cols, names = list(), list()
    
    # input sequence (t-n, ..., t-1)
    for i in range(n_in, 0, -1):
        cols.append(df2.shift(i))
        names += [('input%d(t-%d)')%(j+1, i)for j in range(n_vars)]
    
    # forecast sequence (t, t+1, ..., t+n)
    for i in range(n_out):
        cols.append(df2.shift(-i))
        if i == 0:
            names += [('output%d(t)')%(j+1) for j in range(n_vars)]
        else:
            names += [('output%d(t+%d)')%(j+1, i) for j in range(n_vars)]

    # put it all together
    agg = pd.concat(cols, axis=1)
    agg.columns = names
    
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg

# frame as supervised learning
reframed = series_to_supervised(scaled, 1, 1)

# drop columns we don't want to predict

reframed

In [ ]:

# drop columns we don't want to predict
reframed.drop(reframed.columns[[10,11,12,13,15,16,17,18,19]], axis=1, inplace=True)
print(reframed.head())

In [ ]:
reframed.columns

In [ ]:
# split into train and test sets
values = reframed.values
train = values[:54, :]
test = values[54:, :]

# split into input and outputs
train_X, train_y = train[:, :-1], train[:, -1]
test_X, test_y = test[:, :-1], test[:, -1]

# reshape input to be 3D [samples, timesteps, features]
train_X = train_X.reshape((train_X.shape[0], 1, train_X.shape[1]))
test_X = test_X.reshape((test_X.shape[0], 1, test_X.shape[1]))
print(train_X.shape, train_y.shape, test_X.shape, test_y.shape)

In [ ]:
from matplotlib import pyplot
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout

# design network
model = Sequential()
model.add(LSTM(50, input_shape=(train_X.shape[1], train_X.shape[2])))
model.add(Dropout(0.3))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')

# fit network
history = model.fit(train_X, train_y, epochs=100, batch_size=12, validation_data=(test_X, test_y), verbose=2, shuffle=False)

# plot history
pyplot.plot(history.history['loss'], label='train')
pyplot.plot(history.history['val_loss'], label='test')
pyplot.legend()
pyplot.show()

In [ ]:
 # design network
model = Sequential()
model.add(LSTM(50, input_shape=(train_X.shape[1], train_X.shape[2])))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')

# fit network
history = model.fit(train_X, train_y, epochs=80, batch_size=10, validation_data=(test_X, test_y), verbose=2, shuffle=False)
# plot history
pyplot.plot(history.history['loss'], label='train')
pyplot.plot(history.history['val_loss'], label='test')
pyplot.legend()

In [ ]:
from matplotlib import pyplot
from keras.models import Sequential
from keras.layers import Dense, LSTM

# design network
model = Sequential()
model.add(LSTM(100, input_shape=(train_X.shape[1], train_X.shape[2])))
model.add(Dense(1))
model.compile(loss='mae', optimizer='adam')

# fit network
history = model.fit(train_X, train_y, epochs=500, batch_size=4, validation_data=(test_X, test_y), verbose=2, shuffle=False)

# plot history
pyplot.plot(history.history['loss'], label='train')
pyplot.plot(history.history['val_loss'], label='test')
pyplot.legend()
pyplot.show()
 
# design network
model = Sequential()
model.add(LSTM(100, input_shape=(train_X.shape[1], train_X.shape[2])))
model.add(Dense(1))
model.compile(loss='mae', optimizer='adam')

# fit network
history = model.fit(train_X, train_y, epochs=500, batch_size=4, validation_data=(test_X, test_y), verbose=2, shuffle=False)
# plot history
pyplot.plot(history.history['loss'], label='train')
pyplot.plot(history.history['val_loss'], label='test')
pyplot.legend()

In [ ]:
df = get_table()
import chinese_calendar

for i in range(1, 11):
    df[str(i)].set_index(['time'], inplace=True)

In [ ]:
for i in range(1, 11):
    for j in date_list:
        if j not in df[str(i)].index:
            date_str = str(j)
            date_str = datetime.date(*map(int, date_str.split("/")))
            if chinese_calendar.is_workday(date_str):
                df[str(i)].loc[j] = [0,0,0,0,0]
    print(df[str(i)].shape)

In [ ]:
df['1'].sort_index(inplace=True)

In [ ]:
for i in range(1,11):
    df[str(i)].index = pd.to_datetime(df[str(i)].index)
    df[str(i)].sort_index(inplace=True)
    df[str(i)].index = df[str(i)].index.map(lambda x: x.strftime('%Y-%m-%d'))
    df[str(i)].sort_index(inplace=True)

In [ ]:
df['1'].loc['2']

In [ ]:
type(df['1'].index[1])

## Build Model

In [ ]:
table = read_table()
np.set_printoptions(suppress=True)
np.set_printoptions(suppress=True)
price = pd.DataFrame(columns=list(range(1, 11)))
for i in range(1, 11):
    price.iloc[:, i-1] =(pd.Series(np.log(table[str(i)].open)-np.log(table[str(i)].open.shift())).diff(1).dropna(axis=0, how=any))

In [ ]:
graph = []

graph

In [ ]:

g1

In [ ]:
w = []
for i in g1.index:
    w1 = g1.diff_std[i]*2
    w2 = g1.oc_ratio_mean[i]*2
    w3 = g1.exp_std[i]*6
    print(w1,w2,w3)
    w.append(w1+w2+w3)
print(w)
w = np.array(w)
w = w/w.sum().round(4)
w

# 插值

In [4]:
import numpy as np
import pandas as pd

In [5]:
cols = ['date', 'open', 'high', 'low', 'close', 'volume']
table = {}
for i in range(1, 11):
    num = lambda x: "0"+str(x) if x < 10 else str(x)
    df = pd.read_csv("abc"+num(i)+".csv")
    df.columns = cols
    df.date = pd.to_datetime(df.date)
    df.index = pd.to_datetime(df.date)
    df = df.drop(['date'], axis=1)
    table[str(i)] = df
table

{'1':              open   high    low  close       volume
 date                                               
 2019-01-29  10.96  11.07  10.77  11.00   82663112.0
 2019-01-30  10.95  11.18  10.86  10.95   71200104.0
 2019-01-31  10.98  11.20  10.94  11.10   83162272.0
 2019-02-01  11.20  11.25  10.96  11.20  101427008.0
 2019-02-02   0.00   0.00   0.00   0.00          0.0
 ...           ...    ...    ...    ...          ...
 2020-03-20  12.40  12.68  12.26  12.52  157835296.0
 2020-03-23  12.00  12.35  11.93  12.15  107111360.0
 2020-03-24  12.40  12.68  12.27  12.61  118020024.0
 2020-03-25  12.88  13.07  12.70  12.87  113695776.0
 2020-03-26  12.87  12.87  12.87  12.87          0.0
 
 [287 rows x 5 columns], '2':              open   high    low  close    volume
 date                                            
 2019-01-14  39.91  40.36  39.32  39.45  18716052
 2019-01-15  39.50  41.02  39.33  41.00  40219032
 2019-01-16  41.00  41.46  40.62  41.20  28623404
 2019-01-17  41.20  41.70

In [6]:
table["1"].open.shape

(287,)

In [7]:
table["1"].open.index[-1]

Timestamp('2020-03-26 00:00:00')

In [41]:
from ARMA import run_arma
df = table["10"]
arma_open = run_arma(df.open, df.open.index[0], 500).forecast
arma_close = run_arma(df.close, df.close.index[0], 500).forecast
arma_high = run_arma(df.high, df.high.index[0], 500).forecast
arma_low = run_arma(df.low, df.low.index[0], 500).forecast
arma_volume = run_arma(df.volume, df.volume.index[0], 500).forecast

for i in df[df.open==0].index:
    df.loc[i, ['open']] = arma_open[i]
#     print(df.loc[i, ['open']])
    
for i in df[df.close==0].index:
    df.loc[i, ['close']] = arma_close[i]
    
for i in df[df.high==0].index:
    df.loc[i, ['high']] = arma_high[i]
    
for i in df[df.low==0].index:
    df.loc[i, ['low']] = arma_low[i]
    
for i in df[df.volume==0].index:
    df.loc[i, ['volume']] = arma_volume[i]
df.to_csv("full10.csv")

/opt/anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:492: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  'available', HessianInversionWarning)
/opt/anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:512: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
/opt/anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:492: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  'available', HessianInversionWarning)
/opt/anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:492: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  'available', HessianInversionWarning)
/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/base/tsa_model.py:219: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.

In [26]:
df.loc["2019-02-02",['open']]

open    0.0
Name: 2019-02-02 00:00:00, dtype: float64

In [19]:
j = 10
df = table[str(j)]
arma_open = run_arma(df.open, df.open.index[0], 500).forecast
arma_close = run_arma(df.close, df.close.index[0], 500).forecast
arma_high = run_arma(df.high, df.high.index[0], 500).forecast
arma_low = run_arma(df.low, df.low.index[0], 500).forecast
arma_volume = run_arma(df.volume, df.volume.index[0], 500).forecast

for i in df[df.open==0].index:
    df.loc[i].open = arma_open[i]
for i in df[df.close==0].index:
    df.loc[i].close = arma_close[i]
for i in df[df.high==0].index:
    df.loc[i].high = arma_high[i]
for i in df[df.low==0].index:
    df.loc[i].low = arma_low[i]
for i in df[df.volume==0].index:
    df.loc[i].volume = arma_volume[i]
print(j)
df.to_csv("full10.csv")

/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/kalmanf/kalmanfilter.py:221: RuntimeWarning: divide by zero encountered in true_divide
  Z_mat, R_mat, T_mat)
/opt/anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:492: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  'available', HessianInversionWarning)
/opt/anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:512: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
/opt/anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:492: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  'available', HessianInversionWarning)
/opt/anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:492: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  'available', HessianInversionWarning)
/opt/anaconda3/lib/python3.7/site-pa

10


/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/base/tsa_model.py:219: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)


In [18]:
j = 9
df = table[str(j)]
arma_open = run_arma(df.open, df.open.index[0], 500).forecast
arma_close = run_arma(df.close, df.close.index[0], 500).forecast
arma_high = run_arma(df.high, df.high.index[0], 500).forecast
arma_low = run_arma(df.low, df.low.index[0], 500).forecast
arma_volume = run_arma(df.volume, df.volume.index[0], 500).forecast

for i in df[df.open==0].index:
    df.loc[i].open = arma_open[i]
for i in df[df.close==0].index:
    df.loc[i].close = arma_close[i]
for i in df[df.high==0].index:
    df.loc[i].high = arma_high[i]
for i in df[df.low==0].index:
    df.loc[i].low = arma_low[i]
for i in df[df.volume==0].index:
    df.loc[i].volume = arma_volume[i]
print(j)
df.to_csv("full09.csv")

/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/base/tsa_model.py:219: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)


ValueError: The computed initial AR coefficients are not stationary
You should induce stationarity, choose a different model order, or you can
pass your own start_params.

In [42]:
np.random.rand((-0.07,0.07), [2,1])

TypeError: 'tuple' object cannot be interpreted as an integer

In [43]:
4.11+np.random.rand(1,2)*0.01.round.(2)

array([[0.98490484, 0.59452753]])

In [51]:
np.random.uniform(77113120,79151008,2)

array([78592413.33769564, 78796148.22288905])

In [72]:
np.random.uniform(160165376, 459266208,1)

array([3.55381658e+08])

In [124]:
a = 9045804
b = 30.56
c = 1
for i in np.random.uniform(a, b, c):
    print(i)

30.41765655293114
